In [1]:
import numpy as np
import pandas as pd
#import asyncio
#import aiohttp
import json
import csv
import requests
import time
from cryptodata import get_markets, get_pairs, get_ohlc

In [ ]:
def get_daily_price(asset="btc", api="coinmetrics", data_type="price(usd)", start=0, end=time.time()):
    if (api == "coinmetrics"):
        try:
            url =  "https://coinmetrics.io/api/v1/get_asset_data_for_time_range/"+asset+"/"+data_type
            url = url + "/"+str(int(start)) +"/"+str(int(end))
            r = requests.get(url).json()['result']
            a = pd.DataFrame(r, columns = ['date(utc)',str(asset)+" "+str(data_type)])
            #print(url)
            a[a.columns[0]] = pd.to_datetime(a[a.columns[0]], unit = 's')

            return a
        except:
            print("Error: disfunctional API from Coinmetrics")            
            
    else:
        return 0


def get_assets_from_coimetrics():
    url = 'https://coinmetrics.io/api/v1/get_supported_assets'
    r  = requests.get(url).json()
    return r

def get_whole_market_from_coimetrics():
    assets = get_assets_from_coimetrics()
    market = {}
    for asset in assets:
        market[asset] = get_daily_price(asset)
    return market

def merge_market_data(market, start=Timestamp_from_string('01 01 17')):
    '''merges the market data from the whole market, dumping coins which are too young '''
    assets = list(market.keys())
    #print(assets)
    #print(range(1,len(assets)))
    mkt = market[assets[0]]
    for i in range(1, len(assets)):
        if market[assets[i]]['date(utc)'][0] < start:
            mkt = pd.merge(mkt, market[assets[i]])
    
    return mkt

def rate_of_return(df):
    cols = df.columns
    #print(cols)
    if (type(df[cols[0]][0]) == pd.Timestamp):
        a = df[cols[1:]].values
        b = a/a[0, :]
        df2 = df[cols[0]]
        #df2 = pd.merge(df2, pd.Dataframe(b, columns = cols[1:]))
        #print(b)
        
        return b[-1]
    else:
        print('error return_from_values')

def get_coins_series(coins,start=Timestamp_from_string('01 01 17').timestamp()):
    returns = get_daily_price(coins[0], start=start)
    for i in range(1,len(coins)):
        returns = pd.merge(returns, get_daily_price(coins[i], start=start))
    return returns

 
def genetic_solver(w,cov,mean):
    size = len(mean)
    lr = .0005
    weights = np.reshape(np.tile(w,size),(size,size) )
    s = np.ones((size,size))*lr/2
    t = np.eye(size)*lr*1.5
    q = weights + t - s
    #a  = np.matmul(mean, q)
    a = np.matmul(q, mean)
    old_mean = np.matmul(mean,w)
    new_mean = a[np.argmin(a)]
    new_weight = q[np.argmin(a)]
    dif = (old_mean-new_mean)/old_mean

    if dif < 0.5 and np.abs(w) == w:
        return genetic_solver(new_weight, cov, mean)
    else:
        return new_weight


In [ ]:
# para definir as taxas de base podemos obter os dados do mercado inteiro
market = get_whole_market_from_coimetrics()
# e unir todos a partir de uma dada data
mkt = merge_market_data(market, Timestamp_from_string('01 01 16'))

In [ ]:
# montamos um portfolio com as seguintes moedas:
l = ['btc', 'ltc', 'eth']

#todas as opções podem ser obtidas em get_assets_from_coimetrics()

portfolio = get_coins_series(l)

mu = portfolio.mean()
sigma = portfolio.corr()


In [ ]:
# geramos 1000 porfolios de forma aleatória:
L = 1000
s = np.zeros(L)
m = np.zeros(L)

for i in range(L):
    
    w1 = np.random.rand(3)
    w2 = np.sum(w1)
    w = w1/w2
    m[i] = np.matmul(mu, w)
    s[i] = np.sqrt( np.matmul(np.matmul(w, sigma), w) )


In [ ]:
# fazemos um gráfico desses portfolios aleatórios
import matplotlib.pyplot as plt
plt.scatter(s,m,1,'b')
plt.ylabel('returns')
plt.xlabel('risk')
plt.show()

In [ ]:
# para obter o portfolio de menor risco vamos usar a função ggenetic_solvertic_solver()
# mas ainda não está correta
z = genetic_solver([.4, .4, .2], sigma, mu)
z